# 1: Data Generation for RDD Analysis

## Objective

Generate synthetic e-commerce shopping session data to learn Regression Discontinuity Design fundamentals.

## Setup

**Running Variable:** Cart value (€0-200)  
**Cutoff:** €50 for free shipping eligibility  
**Treatment:** Free shipping (€0) vs standard shipping (€5.95)  
**Outcome:** Purchase completion (binary)  
**True Treatment Effect:** 8 percentage points  

## Generate Data

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import sys
sys.path.append('../src')
from generate_data import generate_rdd_ecommerce_data

# Generate data
df = generate_rdd_ecommerce_data(
    n_sessions=10000,
    cutoff=50.0,
    shipping_cost=5.95,
    treatment_effect=0.08,  # 8 percentage points
    random_seed=14
)


## Quick Summary Statistics

In [2]:

print(f"Data generated: {len(df)} shopping sessions")
print(f"\nTreatment split:")
print(df['treatment'].value_counts())
print(f"\nCompletion rates:")
print(df.groupby('treatment')['completed_purchase'].mean())




Data generated: 10000 shopping sessions

Treatment split:
treatment
1    5672
0    4328
Name: count, dtype: int64

Completion rates:
treatment
0    0.448475
1    0.581805
Name: completed_purchase, dtype: float64


In [3]:
print("Treatment distribution:")
print(df['treatment'].value_counts())
print("\nPurchase completion by treatment:")
print(df.groupby('treatment')['completed_purchase'].agg(['mean', 'sum', 'count']))
print("\nCart value distribution:")
print(df['cart_value'].describe())
print("\nProduct category distribution:")
print(df['product_category'].value_counts())

Treatment distribution:
treatment
1    5672
0    4328
Name: count, dtype: int64

Purchase completion by treatment:
               mean   sum  count
treatment                       
0          0.448475  1941   4328
1          0.581805  3300   5672

Cart value distribution:
count    10000.000000
mean        58.468584
std         27.558560
min          5.000000
25%         38.835000
50%         54.590000
75%         73.742500
max        200.000000
Name: cart_value, dtype: float64

Product category distribution:
product_category
Fashion              2948
Electronics          2511
Home & Garden        2014
Books & Media        1534
Sports & Outdoors     993
Name: count, dtype: int64


## Check Density Around Threshold

In [4]:
# How many sessions are near the €50 threshold?
window = df[(df['cart_value'] >= 40) & (df['cart_value'] <= 60)]
print(f"Sessions with cart value €40-60: {len(window)} ({len(window)/len(df)*100:.1f}%)")
print(f"\nTreatment distribution in this window:")
print(window['treatment'].value_counts())



Sessions with cart value €40-60: 3134 (31.3%)

Treatment distribution in this window:
treatment
0    1647
1    1487
Name: count, dtype: int64


## Save Data

In [5]:
# Save to CSV
df.to_csv('../data/rdd_ecommerce.csv', index=False)
print("Data saved to ../data/rdd_ecommerce.csv")

Data saved to ../data/rdd_ecommerce.csv


# Exploration

In [ ]:
def create_rdd_binned_plot(df, running_var='cart_value', outcome='completed_purchase', 
                           cutoff=50.0, bin_width=2.0):
    """
    Create binned scatter plot showing the RDD discontinuity.
    
    This is the canonical RDD visualization: bin the data by running variable,
    calculate mean outcome in each bin, and plot to visualize the jump at cutoff.
    """
    # Create bins
    bins = np.arange(df[running_var].min(), df[running_var].max() + bin_width, bin_width)
    df['bin'] = pd.cut(df[running_var], bins=bins, labels=bins[:-1] + bin_width/2)
    
    # Calculate mean outcome by bin
    binned = df.groupby(['bin', 'treatment'])[outcome].agg(['mean', 'count']).reset_index()
    binned['bin'] = binned['bin'].astype(float)
    
    # Separate control and treatment
    control = binned[binned['treatment'] == 0]
    treated = binned[binned['treatment'] == 1]
    
    # Create plot
    fig = go.Figure()
    
    # Control group (below cutoff)
    fig.add_trace(go.Scatter(
        x=control['bin'],
        y=control['mean'],
        mode='markers',
        name='Below €50 (Paid Shipping)',
        marker=dict(size=8, color='red', opacity=0.7),
        hovertemplate='Cart: €%{x:.0f}<br>Completion: %{y:.1%}<extra></extra>'
    ))
    
    # Treatment group (at/above cutoff)
    fig.add_trace(go.Scatter(
        x=treated['bin'],
        y=treated['mean'],
        mode='markers',
        name='At/Above €50 (Free Shipping)',
        marker=dict(size=8, color='blue', opacity=0.7),
        hovertemplate='Cart: €%{x:.0f}<br>Completion: %{y:.1%}<extra></extra>'
    ))
    
    # Add vertical line at cutoff
    fig.add_vline(x=cutoff, line_dash="dash", line_color="gray", 
                  annotation_text="€50 Free Shipping Threshold",
                  annotation_position="top")
    
    # Formatting
    fig.update_layout(
        title='RDD Plot: Purchase Completion by Cart Value',
        xaxis_title='Cart Value (€)',
        yaxis_title='Purchase Completion Rate',
        hovermode='closest',
        height=500,
        showlegend=True,

        
        legend=dict(x=0.02, y=0.98),
        font=dict(size=12)
    )
    
    # Format y-axis as percentage
    fig.update_yaxes(tickformat='.0%')
    
    return fig

# Create the plot
fig = create_rdd_binned_plot(df, cutoff=50.0, bin_width=2.0)
fig.show()

/var/folders/s9/3vxnll4s24b0zx1_rpkbk7r40000gp/T/ipykernel_74914/565818134.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned = df.groupby(['bin', 'treatment'])[outcome].agg(['mean', 'count']).reset_index()


In [17]:
def create_rdd_plot_with_fits(df, cutoff=50.0, bandwidth=20.0):
    """
    Create RDD plot with local linear regression fits on both sides of cutoff.
    """
    from sklearn.linear_model import LinearRegression
    
    # Filter to bandwidth around cutoff
    df_window = df[(df['cart_value'] >= cutoff - bandwidth) & 
                   (df['cart_value'] <= cutoff + bandwidth)].copy()
    
    # Separate control and treatment
    control = df_window[df_window['treatment'] == 0]
    treated = df_window[df_window['treatment'] == 1]
    
    # Fit linear models
    if len(control) > 0:
        X_control = control['cart_value'].values.reshape(-1, 1)
        y_control = control['completed_purchase'].values
        model_control = LinearRegression().fit(X_control, y_control)
        
        # Predict for smooth line
        x_control_line = np.linspace(control['cart_value'].min(), cutoff, 100)
        y_control_pred = model_control.predict(x_control_line.reshape(-1, 1))
    
    if len(treated) > 0:
        X_treated = treated['cart_value'].values.reshape(-1, 1)
        y_treated = treated['completed_purchase'].values
        model_treated = LinearRegression().fit(X_treated, y_treated)
        
        # Predict for smooth line
        x_treated_line = np.linspace(cutoff, treated['cart_value'].max(), 100)
        y_treated_pred = model_treated.predict(x_treated_line.reshape(-1, 1))
    
    # Create binned data for scatter
    bins = np.arange(cutoff - bandwidth, cutoff + bandwidth + 2, 2)
    df_window['bin'] = pd.cut(df_window['cart_value'], bins=bins, 
                               labels=bins[:-1] + 1)
    binned = df_window.groupby(['bin', 'treatment'], observed=False)['completed_purchase'].mean().reset_index()
    binned['bin'] = binned['bin'].astype(float)
    
    control_binned = binned[binned['treatment'] == 0]
    treated_binned = binned[binned['treatment'] == 1]
    
    # Create plot
    fig = go.Figure()
    
    # Scatter points
    fig.add_trace(go.Scatter(
        x=control_binned['bin'],
        y=control_binned['completed_purchase'],
        mode='markers',
        name='Control (Paid Shipping)',
        marker=dict(size=10, color='red', opacity=0.6),
        showlegend=True
    ))
    
    fig.add_trace(go.Scatter(
        x=treated_binned['bin'],
        y=treated_binned['completed_purchase'],
        mode='markers',
        name='Treatment (Free Shipping)',
        marker=dict(size=10, color='blue', opacity=0.6),
        showlegend=True
    ))
    
    # Fitted lines
    fig.add_trace(go.Scatter(
        x=x_control_line,
        y=y_control_pred,
        mode='lines',
        name='Control Fit',
        line=dict(color='red', width=3),
        showlegend=False
    ))
    
    fig.add_trace(go.Scatter(
        x=x_treated_line,
        y=y_treated_pred,
        mode='lines',
        name='Treatment Fit',
        line=dict(color='blue', width=3),
        showlegend=False
    ))
    
    # Add cutoff line
    fig.add_vline(x=cutoff, line_dash="dash", line_color="gray", line_width=2)
    
    # Calculate treatment effect at cutoff
    y_control_at_cutoff = model_control.predict([[cutoff]])[0]
    y_treated_at_cutoff = model_treated.predict([[cutoff]])[0]
    treatment_effect = y_treated_at_cutoff - y_control_at_cutoff
    
    # Add annotation showing the jump
    fig.add_annotation(
        x=cutoff,
        y=(y_control_at_cutoff + y_treated_at_cutoff) / 2,
        text=f"Jump: {treatment_effect:.1%}",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="black",
        ax=1.25*cutoff,
        ay=0,
        font=dict(size=14, color="black"),
        bgcolor="yellow",
        opacity=0.8
    )
    
    fig.update_layout(
        title=f'RDD Plot with Local Linear Fits (Bandwidth = €{bandwidth})',
        xaxis_title='Cart Value (€)',
        yaxis_title='Purchase Completion Rate',
        height=550,
        hovermode='closest',
        font=dict(size=12)
    )
    
    fig.update_yaxes(tickformat='.0%')
    
    print(f"\n📊 RDD Estimate:")
    print(f"Control completion at €50: {y_control_at_cutoff:.1%}")
    print(f"Treatment completion at €50: {y_treated_at_cutoff:.1%}")
    print(f"Treatment Effect: {treatment_effect:.1%} ({treatment_effect*100:.1f} percentage points)")
    print(f"\nTrue effect we set: 8.0 percentage points")
    print(f"Our estimate: {treatment_effect*100:.1f} percentage points")
    print(f"Difference: {abs(treatment_effect*100 - 8.0):.1f} pp")
    
    return fig

# Create plot with fits
fig = create_rdd_plot_with_fits(df, cutoff=50.0, bandwidth=20.0)
fig.show()


📊 RDD Estimate:
Control completion at €50: 47.9%
Treatment completion at €50: 54.7%
Treatment Effect: 6.8% (6.8 percentage points)

True effect we set: 8.0 percentage points
Our estimate: 6.8 percentage points
Difference: 1.2 pp


In [8]:
def check_covariate_balance(df, cutoff=50.0, bandwidth=20.0):
    """
    Check if covariates are balanced (smooth) at the cutoff.
    
    If RDD is valid, customer characteristics should NOT jump at €50.
    Only the treatment (free shipping) should jump.
    """
    # Filter to bandwidth
    df_window = df[(df['cart_value'] >= cutoff - bandwidth) & 
                   (df['cart_value'] <= cutoff + bandwidth)].copy()
    
    # Covariates to check
    covariates = ['previous_purchases', 'account_tenure_days', 'items_in_cart']
    
    fig = make_subplots(rows=1, cols=3, subplot_titles=covariates)
    
    for i, covar in enumerate(covariates, 1):
        # Bin the data
        bins = np.arange(cutoff - bandwidth, cutoff + bandwidth + 2, 2)
        df_window['bin'] = pd.cut(df_window['cart_value'], bins=bins, 
                                   labels=bins[:-1] + 1)
        
        binned = df_window.groupby(['bin', 'treatment'], observed=False)[covar].mean().reset_index()
        binned['bin'] = binned['bin'].astype(float)
        
        control = binned[binned['treatment'] == 0]
        treated = binned[binned['treatment'] == 1]
        
        # Add traces
        fig.add_trace(
            go.Scatter(x=control['bin'], y=control[covar], 
                      mode='markers', marker=dict(color='red', size=8),
                      name='Control', showlegend=(i==1)),
            row=1, col=i
        )
        
        fig.add_trace(
            go.Scatter(x=treated['bin'], y=treated[covar],
                      mode='markers', marker=dict(color='blue', size=8),
                      name='Treatment', showlegend=(i==1)),
            row=1, col=i
        )
        
        # Add cutoff line
        fig.add_vline(x=cutoff, line_dash="dash", line_color="gray",
                     row=1, col=i)
    
    fig.update_layout(height=400, title_text="Covariate Balance Check: Should be SMOOTH at €50")
    fig.update_xaxes(title_text="Cart Value (€)")
    
    # Statistical test
    print("📊 Covariate Balance Tests at Cutoff:\n")
    
    df_near_cutoff = df[(df['cart_value'] >= cutoff - 5) & 
                        (df['cart_value'] <= cutoff + 5)]
    
    control_near = df_near_cutoff[df_near_cutoff['treatment'] == 0]
    treated_near = df_near_cutoff[df_near_cutoff['treatment'] == 1]
    
    for covar in covariates:
        control_mean = control_near[covar].mean()
        treated_mean = treated_near[covar].mean()
        diff = treated_mean - control_mean
        
        # Simple t-test
        from scipy.stats import ttest_ind
        t_stat, p_value = ttest_ind(control_near[covar], treated_near[covar])
        
        print(f"{covar}:")
        print(f"  Control (€45-50): {control_mean:.2f}")
        print(f"  Treatment (€50-55): {treated_mean:.2f}")
        print(f"  Difference: {diff:.2f}")
        print(f"  p-value: {p_value:.3f} {'✓ BALANCED' if p_value > 0.05 else '✗ IMBALANCED'}\n")
    
    return fig

# Check balance
fig = check_covariate_balance(df, cutoff=50.0, bandwidth=20.0)
fig.show()

📊 Covariate Balance Tests at Cutoff:

previous_purchases:
  Control (€45-50): 2.46
  Treatment (€50-55): 2.18
  Difference: -0.28
  p-value: 0.043 ✗ IMBALANCED

account_tenure_days:
  Control (€45-50): 208.50
  Treatment (€50-55): 201.83
  Difference: -6.66
  p-value: 0.364 ✓ BALANCED

items_in_cart:
  Control (€45-50): 4.08
  Treatment (€50-55): 4.04
  Difference: -0.04
  p-value: 0.619 ✓ BALANCED



In [9]:
def check_manipulation_density(df, cutoff=50.0):
    """
    McCrary density test: Check if cart values are suspiciously bunched at cutoff.
    
    If customers are gaming the system, we'd see excess density right at €50.
    """
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=['Histogram of Cart Values', 
                                       'Smoothed Density'])
    
    # Histogram
    fig.add_trace(
        go.Histogram(x=df['cart_value'], nbinsx=50, 
                    marker_color='lightblue', name='All Carts'),
        row=1, col=1
    )
    
    fig.add_vline(x=cutoff, line_dash="dash", line_color="red", 
                 annotation_text="€50 Threshold", row=1, col=1)
    
    # Kernel density estimation
    from scipy.stats import gaussian_kde
    
    # Separate control and treatment for density
    control = df[df['treatment'] == 0]['cart_value']
    treated = df[df['treatment'] == 1]['cart_value']
    
    # Create smooth density estimates
    x_range = np.linspace(df['cart_value'].min(), df['cart_value'].max(), 500)
    
    if len(control) > 50:
        kde_control = gaussian_kde(control, bw_method=0.3)
        density_control = kde_control(x_range[x_range < cutoff])
        
        fig.add_trace(
            go.Scatter(x=x_range[x_range < cutoff], y=density_control,
                      mode='lines', name='Control Density',
                      line=dict(color='red', width=2)),
            row=1, col=2
        )
    
    if len(treated) > 50:
        kde_treated = gaussian_kde(treated, bw_method=0.3)
        density_treated = kde_treated(x_range[x_range >= cutoff])
        
        fig.add_trace(
            go.Scatter(x=x_range[x_range >= cutoff], y=density_treated,
                      mode='lines', name='Treatment Density',
                      line=dict(color='blue', width=2)),
            row=1, col=2
        )
    
    fig.add_vline(x=cutoff, line_dash="dash", line_color="red", row=1, col=2)
    
    fig.update_xaxes(title_text="Cart Value (€)")
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Density", row=1, col=2)
    
    fig.update_layout(height=400, showlegend=True,
                     title_text="Manipulation Test: Looking for Bunching at €50")
    
    # Statistical check: compare density just below vs just above
    just_below = df[(df['cart_value'] >= cutoff - 2) & (df['cart_value'] < cutoff)]
    just_above = df[(df['cart_value'] >= cutoff) & (df['cart_value'] < cutoff + 2)]
    
    print("📊 Manipulation Test:")
    print(f"\nCarts in €48-50: {len(just_below)}")
    print(f"Carts in €50-52: {len(just_above)}")
    print(f"Ratio: {len(just_above) / len(just_below):.2f}")
    print("\nInterpretation:")
    print("- Ratio ≈ 1.0: No evidence of manipulation ✓")
    print("- Ratio > 1.3: Possible bunching at threshold ⚠️")
    print("- Ratio > 1.5: Strong evidence of manipulation ✗")
    
    # Check for exact €50.00 values
    exact_50 = len(df[df['cart_value'] == 50.0])
    print(f"\nCarts at exactly €50.00: {exact_50}")
    print(f"Percentage of all carts: {exact_50/len(df)*100:.2f}%")
    
    if exact_50 / len(df) > 0.02:  # More than 2% at exact cutoff
        print("⚠️ Warning: Suspicious number of carts at exact cutoff")
    else:
        print("✓ No suspicious bunching at exact cutoff")
    
    return fig

# Check for manipulation
fig = check_manipulation_density(df, cutoff=50.0)
fig.show()

📊 Manipulation Test:

Carts in €48-50: 350
Carts in €50-52: 308
Ratio: 0.88

Interpretation:
- Ratio ≈ 1.0: No evidence of manipulation ✓
- Ratio > 1.3: Possible bunching at threshold ⚠️
- Ratio > 1.5: Strong evidence of manipulation ✗

Carts at exactly €50.00: 3
Percentage of all carts: 0.03%
✓ No suspicious bunching at exact cutoff
